In [1]:
%load_ext autoreload
%autoreload 2
import pytz
from datetime import datetime, timedelta, date
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from src.inferencesm import load_predictions_from_store

In [2]:



# Define Argentina's timezone (GMT-3)
argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')

# Get the current date and time in Argentina
current_time_in_argentina = datetime.now(argentina_tz).replace(minute=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
# Round down (floor) to the nearest hour by setting minutes, seconds, and microseconds to 0
current_date = pd.to_datetime(current_time_in_argentina)
current_date

Timestamp('2024-10-05 20:00:00')

In [3]:
from src.inferencesm import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

# Create a label encoder object
label_encoder = LabelEncoder()

# Apply label encoding to 'line' and 'station'
features['line'] = label_encoder.fit_transform(features['line'])
features['station'] = label_encoder.fit_transform(features['station'])

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/603286
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2024-09-21 19:00:00+00:00 to 2024-10-05 19:00:00+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.75s) 
Index(['station', 'line'], dtype='object')


In [4]:
features

,total_pax_previous_336_hour,total_pax_previous_335_hour,total_pax_previous_334_hour,total_pax_previous_333_hour,total_pax_previous_332_hour,total_pax_previous_331_hour,total_pax_previous_330_hour,total_pax_previous_329_hour,total_pax_previous_328_hour,total_pax_previous_327_hour,...,total_pax_previous_7_hour,total_pax_previous_6_hour,total_pax_previous_5_hour,total_pax_previous_4_hour,total_pax_previous_3_hour,total_pax_previous_2_hour,total_pax_previous_1_hour,hour_of_entry,station,line
18,469.0,574.0,233.0,193.0,41.0,0.0,0.0,0.0,0.0,0.0,...,1020.0,689.0,885.0,549.0,873.0,598.0,1078.0,2024-10-05 20:00:00+00:00,0,0
7,190.0,219.0,75.0,76.0,13.0,0.0,0.0,0.0,0.0,0.0,...,391.0,282.0,410.0,244.0,525.0,329.0,544.0,2024-10-05 20:00:00+00:00,1,0
24,585.0,773.0,286.0,245.0,36.0,0.0,0.0,0.0,0.0,0.0,...,1146.0,806.0,1073.0,603.0,1138.0,774.0,1222.0,2024-10-05 20:00:00+00:00,2,1
2,167.0,279.0,141.0,94.0,15.0,0.0,0.0,0.0,0.0,0.0,...,283.0,197.0,371.0,296.0,514.0,416.0,533.0,2024-10-05 20:00:00+00:00,3,2
19,699.0,834.0,316.0,442.0,117.0,0.0,0.0,0.0,0.0,0.0,...,1153.0,938.0,1124.0,667.0,1516.0,809.0,1331.0,2024-10-05 20:00:00+00:00,4,1
27,346.0,390.0,118.0,128.0,9.0,0.0,0.0,0.0,0.0,0.0,...,742.0,511.0,652.0,326.0,755.0,489.0,795.0,2024-10-05 20:00:00+00:00,5,0
35,883.0,1342.0,601.0,718.0,112.0,0.0,0.0,0.0,0.0,0.0,...,873.0,679.0,1082.0,634.0,1438.0,904.0,1457.0,2024-10-05 20:00:00+00:00,6,1
15,1049.0,1248.0,615.0,716.0,212.0,0.0,0.0,0.0,0.0,0.0,...,931.0,668.0,1291.0,820.0,1586.0,1155.0,1836.0,2024-10-05 20:00:00+00:00,7,1
0,324.0,434.0,172.0,170.0,28.0,0.0,0.0,0.0,0.0,0.0,...,631.0,427.0,734.0,393.0,822.0,535.0,761.0,2024-10-05 20:00:00+00:00,8,0
1,507.0,705.0,232.0,231.0,78.0,0.0,0.0,0.0,0.0,0.0,...,1055.0,724.0,1115.0,751.0,1401.0,964.0,1249.0,2024-10-05 20:00:00+00:00,9,0


In [5]:
from src.model_registry import get_latest_model_from_registry
from src.inferencesm import get_model_predictions

model = get_latest_model_from_registry(model_name='subwayBA_passenger_flow_updt', status= 'Production')
predictions = get_model_predictions(model, features)

2024-10-05 20:41:10,730 WARNING: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html

2024-10-05 20:41:11,420 WARNING: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('ruamel')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages

2024-10-05 20:41:17,690 WARNING: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x

2024-10-05 20:41:17,710 WARNING: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead



COMET WARNING: This method has been deprecated, we recommend using the api.get_model(...) method to get the Model Object and then using model.download(...)
COMET INFO: Downloading registry model 'subwayBA_passenger_flow_updt', version '1.1.0', stage None from workspace 'simonmontt'...
COMET INFO: Unzipping model to 'C:\\Users\\PC\\Documents\\Proyect\\passenger_flow_predictor\\models' ...
COMET INFO: done!


2024-10-05 20:41:22,350 WARNING: DeprecationWarning: CountryHoliday is deprecated, use country_holidays instead.

2024-10-05 20:41:22,580 WARNING: InconsistentVersionWarning: Trying to unpickle estimator FunctionTransformer from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

2024-10-05 20:41:22,620 WARNING: InconsistentVersionWarning: Trying to unpickle estimator MultiOutputRegressor from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

2024-10-05 20:41:22,620 WARNING: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.2.2 when using version 1.5.2. This might lead to breaking code 

In [6]:
predictions['hour_of_entry'] = current_date
predictions

,total_pax_next_1_hour,total_pax_next_2_hour,total_pax_next_3_hour,station,line,hour_of_entry
0,510.0,646.0,212.0,0,0,2024-10-05 20:00:00
1,279.0,293.0,96.0,1,0,2024-10-05 20:00:00
2,613.0,752.0,276.0,2,1,2024-10-05 20:00:00
3,269.0,362.0,126.0,3,2,2024-10-05 20:00:00
4,781.0,949.0,418.0,4,1,2024-10-05 20:00:00
5,320.0,366.0,127.0,5,0,2024-10-05 20:00:00
6,828.0,1167.0,516.0,6,1,2024-10-05 20:00:00
7,808.0,987.0,420.0,7,1,2024-10-05 20:00:00
8,338.0,457.0,165.0,8,0,2024-10-05 20:00:00
9,1013.0,1230.0,458.0,9,0,2024-10-05 20:00:00


In [7]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generated for the next 3 hours by our production model",
    primary_key = ['station', 'line', 'hour_of_entry'],
    event_time='hour_of_entry',
)

Connected. Call `.close()` to terminate connection gracefully.


In [8]:
predictions['station'] = predictions['station'].astype('int32')
predictions['line'] = predictions['line'].astype('int32')
print(predictions.dtypes)
feature_group.insert(predictions, write_options={"wait_for_job": False})

total_pax_next_1_hour           float64
total_pax_next_2_hour           float64
total_pax_next_3_hour           float64
station                           int32
line                              int32
hour_of_entry            datetime64[ns]
dtype: object


Uploading Dataframe: 0.00% |          | Rows 0/44 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group__1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/603286/jobs/named/model_predictions_feature_group__1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1882e9f5fa0>, None)